# creating the graph database

here we recreate the send receive list (its a little redundant, so i will skip talking about the stuff that's done in the previous dataset) and then use that plus some string manipulation to generate the necessary cypher commands to instantiate the database

##### NOTE: we used the free version of neo4j to create the graph database, it has pretty limited processing capability. we were able to generate all the nodes in one query, but we had the best results copying the the edge queries over about 20 to 40 at a time and letting it process in batches.

##### NOTE 2: the graph database is a little messy, some redundencies with nodes are included in the setup. that said, we were able to get the information we needed with the graph db as is, so we didnt fool around with it too much and hid nodes that were irrelevant

### import statements

once again, we really only needed python and some join logic to process this data into the graph database queries

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 20)

step 1: finding the missing exectives within the company records so that we can start to find out who they were interacting with

names to filter the dataframe on were taken from article 460 naming the missing executives

In [3]:
missing_execs = ['Sten Sanjorge', 'Ingrid Precipice', 'Field-Corrente' , 'Rasgueo of Orhan', 'Willem Basoue-Country']

In [4]:
missing_execs_lastOnly = ['Sanjorge', 'Ingrid', 'Precipice', 'Corrente' , 'Rasgueo', 'Orhan', 'Willem', 'Basoue-Country']

In [5]:
try:
    top_level = pd.read_excel('C:/Users/Andy/PycharmProjects/pythonProject/The_Last_Stand/TP-1_Kronos/EmployeeRecords.xlsx')
except:
    top_level = pd.read_excel('C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/EmployeeRecords.xlsx')

display(top_level.head())

,LastName,FirstName,BirthDate,BirthCountry,Gender,CitizenshipCountry,CitizenshipBasis,CitizenshipStartDate,PassportCountry,PassportIssueDate,PassportExpirationDate,CurrentEmploymentType,CurrentEmploymentTitle,CurrentEmploymentStartDate,EmailAddress,MilitaryServiceBranch,MilitaryDischargeType,MilitaryDischargeDate
0,Mies Haber,Ruscella,1964-04-26,Kronos,Female,Kronos,BirthNation,1964-04-26,NaN,NaT,NaT,Administration,Assistant to Engineering Group Manager,2003-04-02,Ruscella.Mies.Haber@gastech.com.kronos,ArmedForcesOfKronos,HonorableDischarge,1984-10-01
1,Fusil,Stenig,1963-04-29,Tethys,Male,Tethys,BirthNation,1963-04-29,Tethys,2006-04-03,2016-04-02,Security,Building Control,2004-02-05,Stenig.Fusil@gastech.com.kronos,TethanDefenseForceArmy,HonorableDischarge,1987-10-01
2,Balas,Felix,1960-03-04,Tethys,Male,Tethys,BirthNation,1960-03-04,Tethys,2005-04-15,2015-04-14,Engineering,Engineer,2004-09-02,Felix.Balas@gastech.com.kronos,TethanDefenseForceNavy,HonorableDischarge,1988-10-01
3,Coginian,Dante,1971-12-01,Kronos,Male,Kronos,BirthNation,1971-12-01,NaN,NaT,NaT,Facilities,Lead Janitor,1995-01-07,Dante.Coginian@gastech.com.kronos,ArmedForcesOfKronos,GeneralDischarge,1992-10-01
4,Ovan,Bertrand,1964-12-12,Tethys,Male,Tethys,BirthNation,1964-12-12,Tethys,2008-10-03,2018-10-02,Facilities,Facilities Group Manager,1998-10-01,Bertrand.Ovan@gastech.com.kronos,TethanDefenseForceAir,HonorableDischarge,1993-10-01


In [6]:
mask = top_level['LastName'].str.contains(missing_execs_lastOnly[0])
for val in missing_execs_lastOnly[1:]:
    mask |= top_level['LastName'].str.contains(val)
    mask |= top_level['FirstName'].str.contains(val)
filtered_top_level = top_level[mask]
display(filtered_top_level)

# filtered_top_level.to_csv('victims_frame.csv')

,LastName,FirstName,BirthDate,BirthCountry,Gender,CitizenshipCountry,CitizenshipBasis,CitizenshipStartDate,PassportCountry,PassportIssueDate,PassportExpirationDate,CurrentEmploymentType,CurrentEmploymentTitle,CurrentEmploymentStartDate,EmailAddress,MilitaryServiceBranch,MilitaryDischargeType,MilitaryDischargeDate
27,Strum,Orhan,1963-10-18,Asteria,Male,Tethys,ParentCitizenship,1963-10-18,Tethys,2011-06-02,2021-06-01,Executive,SVP/COO,2003-08-07,Orhan.Strum@gastech.com.kronos,NaN,NaN,NaT
43,Vasco-Pais,Willem,1975-03-18,Tethys,Male,Tethys,BirthNation,1975-03-18,Tethys,2011-11-26,2021-11-25,Executive,Environmental Safety Advisor,2010-06-05,Willem.Vasco-Pais@gastech.com.kronos,NaN,NaN,NaT
44,Sanjorge Jr.,Sten,1948-08-29,Tethys,Male,Tethys,BirthNation,1948-08-29,Tethys,2013-04-21,2023-04-20,Executive,President/CEO,1990-11-09,Sten.Sanjorge Jr.@gastech.com.kronos,NaN,NaN,NaT
45,Barranco,Ingrid,1961-10-26,Tethys,Female,Tethys,BirthNation,1961-10-26,Tethys,2013-07-01,2023-06-30,Executive,SVP/CFO,2003-09-30,Ingrid.Barranco@gastech.com.kronos,NaN,NaN,NaT
46,Campo-Corrente,Ada,1951-11-26,Tethys,Female,Tethys,BirthNation,1951-11-26,Tethys,2013-07-14,2023-07-13,Executive,SVP/CIO,2004-10-05,Ada.Campo-Corrente@gastech.com.kronos,NaN,NaN,NaT


names and employment titles are consistent with the names in the article. although the data doesn't match 1:1, job titles, number of returns, and no missing or plural entries would indicate that these are our missing executives.

step 2: looking at the email traffic of the entity emails

In [7]:
try:
    headers = pd.read_csv('C:/Users/Andy/PycharmProjects/pythonProject/The_Last_Stand/TP-1_Kronos/email_headers.csv', encoding='cp1252')
except:
    headers = pd.read_csv('C:/Users/andyt/PycharmProjects/classworker_General/The_Last_Stand/TP-1_Kronos/email_headers.csv', encoding='cp1252')

# display(headers.head())


In [8]:
mask = headers['From'].isin(filtered_top_level['EmailAddress'])
outgoing_df = headers[mask]

outgoing_df['To'] = outgoing_df['To'].str.split(',')

# display(outgoing_df)

C:\Users\andyt\AppData\Local\Temp/ipykernel_2732/926598733.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outgoing_df['To'] = outgoing_df['To'].str.split(',')


In [9]:
receivers = outgoing_df.explode('To')
receivers = receivers['To'].drop_duplicates().to_list()
receivers = list(set(receivers)-set(filtered_top_level['EmailAddress'].to_list()))
print(receivers)
print(len(receivers))

[' Felix.Resumir@gastech.com.kronos', ' Vira.Frente@gastech.com.kronos', ' Marin.Onda@gastech.com.kronos', ' Minke.Mies@gastech.com.kronos', ' Axel.Calzas@gastech.com.kronos', ' Emile.Arpa@gastech.com.kronos', ' Gustav.Cazar@gastech.com.kronos', ' Varro.Awelon@gastech.com.kronos', 'Kare.Orilla@gastech.com.kronos', 'Varja.Lagos@gastech.com.kronos', 'Sten.Sanjorge Jr.@gastech.com.tethys', ' Ingrid.Barranco@gastech.com.kronos', ' Nils.Calixto@gastech.com.kronos', ' Orhan.Strum@gastech.com.kronos', 'Stenig.Fusil@gastech.com.kronos', ' Felix.Balas@gastech.com.kronos', ' Sten.Sanjorge Jr.@gastech.com.tethys', ' Dante.Coginian@gastech.com.kronos', ' Stenig.Fusil@gastech.com.kronos', ' Isia.Vann@gastech.com.kronos', ' Albina.Hafon@gastech.com.kronos', ' Linnea.Bergen@gastech.com.kronos', ' Adra.Nubarron@gastech.com.kronos', 'Henk.Mies@gastech.com.kronos', ' Claudio.Hawelon@gastech.com.kronos', ' Henk.Mies@gastech.com.kronos', ' Lucas.Alcazar@gastech.com.kronos', ' Valeria.Morlun@gastech.com.kr

In [10]:
receivers_df = top_level[top_level['EmailAddress'].isin(receivers)]
# display(receivers_df)
print(receivers_df.shape)

(12, 18)


not everyone who was sent an email by one of our original 5 is in the employee records

step 3: people who sent emails to the original 5


In [11]:
incoming_df = headers.copy()
incoming_df['To'] = incoming_df['To'].str.split(',')

mask = incoming_df['To'].apply(lambda x: any(item in x for item in filtered_top_level.EmailAddress.to_list()))
filtered_senders = incoming_df[mask]

# display(filtered_senders)

In [12]:
senders_df = top_level[top_level['EmailAddress'].isin(filtered_senders['From'])]
# display(senders_df)
print(senders_df.shape)

(14, 18)


people who sent an email to one of the five victims


finally combining both lists to generate a list of suspects

In [13]:
suspects_df = pd.concat([receivers_df, senders_df]).drop_duplicates()

suspects_df = suspects_df[~suspects_df['EmailAddress'].isin(filtered_top_level.EmailAddress.to_list())]

display(suspects_df)

,LastName,FirstName,BirthDate,BirthCountry,Gender,CitizenshipCountry,CitizenshipBasis,CitizenshipStartDate,PassportCountry,PassportIssueDate,PassportExpirationDate,CurrentEmploymentType,CurrentEmploymentTitle,CurrentEmploymentStartDate,EmailAddress,MilitaryServiceBranch,MilitaryDischargeType,MilitaryDischargeDate
0,Mies Haber,Ruscella,1964-04-26,Kronos,Female,Kronos,BirthNation,1964-04-26,NaN,NaT,NaT,Administration,Assistant to Engineering Group Manager,2003-04-02,Ruscella.Mies.Haber@gastech.com.kronos,ArmedForcesOfKronos,HonorableDischarge,1984-10-01
1,Fusil,Stenig,1963-04-29,Tethys,Male,Tethys,BirthNation,1963-04-29,Tethys,2006-04-03,2016-04-02,Security,Building Control,2004-02-05,Stenig.Fusil@gastech.com.kronos,TethanDefenseForceArmy,HonorableDischarge,1987-10-01
6,Resumir,Felix,1959-06-30,Tethys,Male,Tethys,BirthNation,1959-06-30,Tethys,2005-03-20,2015-03-19,Security,Security Group Manager,2003-09-15,Felix.Resumir@gastech.com.kronos,TethanDefenseForceArmy,HonorableDischarge,1997-10-01
11,Mies,Henk,1984-09-23,Kronos,Male,Kronos,BirthNation,1984-09-23,NaN,NaT,NaT,Facilities,Truck Driver,2013-07-12,Henk.Mies@gastech.com.kronos,ArmedForcesOfKronos,HonorableDischarge,2004-10-01
13,Nant,Irene,1985-03-13,Kronos,Female,Kronos,BirthNation,1985-03-13,NaN,NaT,NaT,Facilities,Truck Driver,2008-11-30,Irene.Nant@gastech.com.kronos,ArmedForcesOfKronos,GeneralDischarge,2005-10-01
18,Morluniau,Cecilia,1988-09-02,Kronos,Female,Kronos,BirthNation,1988-09-02,NaN,NaT,NaT,Facilities,Truck Driver,2009-04-30,Cecilia.Morluniau@gastech.com.kronos,ArmedForcesOfKronos,GeneralDischarge,2008-10-01
19,Bodrogi,Loreto,1989-04-17,Kronos,Male,Kronos,BirthNation,1989-04-17,NaN,NaT,NaT,Security,Site Control,2013-08-17,Loreto.Bodrogi@gastech.com.kronos,ArmedForcesOfKronos,HonorableDischarge,2008-10-01
26,Ferro,Inga,1989-06-17,Kronos,Female,Kronos,BirthNation,1989-06-17,NaN,NaT,NaT,Security,Site Control,2013-01-11,Inga.Ferro@gastech.com.kronos,ArmedForcesOfKronos,GeneralDischarge,2012-10-01
30,Pantanal,Rachel,1984-08-22,Tethys,Female,Tethys,BirthNation,1984-08-22,Tethys,2013-06-13,2023-06-12,Administration,Assistant to CIO,2013-10-01,Rachel.Pantanal@gastech.com.kronos,NaN,NaN,NaT
35,Orilla,Kare,1964-02-29,Tethys,Female,Tethys,BirthNation,1964-02-29,Tethys,2013-07-01,2023-06-30,Engineering,Drill Technician,2005-11-12,Kare.Orilla@gastech.com.kronos,NaN,NaN,NaT


### step 4: creating the graph database

okay now creating the neo4j query, to kick off, we are taking the emails database and comparing it to our main dataframe of suspects and victims, we want there to be one edge for each email between senders and receivers in those groups, so we are creating a new dataframe to collect all of those emails


In [14]:
send_receive = headers.copy()
send_receive['To'] = send_receive['To'].str.split(',')

processed_SR = pd.DataFrame(columns=["Sender", "Receiver", 'Date', 'Subject'])

for index, row in send_receive.iterrows():
    value_a = row['From']
    for value_b in row['To']:
        new_row = pd.DataFrame({'Sender': [value_a], 
                                'Receiver': [value_b],
                                'Date': [row['Date']],
                                'Subject': [row['Subject']]
                                })
        processed_SR = pd.concat([processed_SR, new_row], ignore_index=True)

# display(processed_SR)

### step 5: creating flags for legibility

now we want to flag each node with a relevant flag to figure out if they are a victim, one hop away, or two hops away. these will go into our final node creation so that its easier to identify who is who when exploring the graph database

In [15]:
processed_SR2 = processed_SR.copy()

processed_SR2['Vic_Flag'] = 0
processed_SR2['1D_Flag'] = 0
processed_SR2['2D_Flag'] = 0

display(processed_SR2)
display(filtered_top_level)
display(suspects_df)

,Sender,Receiver,Date,Subject,Vic_Flag,1D_Flag,2D_Flag
0,Sven.Flecha@gastech.com.kronos,Isak.Baza@gastech.com.kronos,1/6/2014 8:39,GT-SeismicProcessorPro Bug Report,0,0,0
1,Sven.Flecha@gastech.com.kronos,Lucas.Alcazar@gastech.com.kronos,1/6/2014 8:39,GT-SeismicProcessorPro Bug Report,0,0,0
2,Kanon.Herrero@gastech.com.kronos,Felix.Resumir@gastech.com.kronos,1/6/2014 8:58,Inspection request for site,0,0,0
3,Kanon.Herrero@gastech.com.kronos,Hideki.Cocinaro@gastech.com.kronos,1/6/2014 8:58,Inspection request for site,0,0,0
4,Kanon.Herrero@gastech.com.kronos,Inga.Ferro@gastech.com.kronos,1/6/2014 8:58,Inspection request for site,0,0,0
...,...,...,...,...,...,...,...
9058,Albina.Hafon@gastech.com.kronos,Valeria.Morlun@gastech.com.kronos,1/17/2014 20:46,RE: Traffic advisory for today,0,0,0
9059,Albina.Hafon@gastech.com.kronos,Cecilia.Morluniau@gastech.com.kronos,1/17/2014 20:46,RE: Traffic advisory for today,0,0,0
9060,Albina.Hafon@gastech.com.kronos,Henk.Mies@gastech.com.kronos,1/17/2014 20:46,RE: Traffic advisory for today,0,0,0
9061,Albina.Hafon@gastech.com.kronos,Dylan.Scozzese@gastech.com.kronos,1/17/2014 20:46,RE: Traffic advisory for today,0,0,0


,LastName,FirstName,BirthDate,BirthCountry,Gender,CitizenshipCountry,CitizenshipBasis,CitizenshipStartDate,PassportCountry,PassportIssueDate,PassportExpirationDate,CurrentEmploymentType,CurrentEmploymentTitle,CurrentEmploymentStartDate,EmailAddress,MilitaryServiceBranch,MilitaryDischargeType,MilitaryDischargeDate
27,Strum,Orhan,1963-10-18,Asteria,Male,Tethys,ParentCitizenship,1963-10-18,Tethys,2011-06-02,2021-06-01,Executive,SVP/COO,2003-08-07,Orhan.Strum@gastech.com.kronos,NaN,NaN,NaT
43,Vasco-Pais,Willem,1975-03-18,Tethys,Male,Tethys,BirthNation,1975-03-18,Tethys,2011-11-26,2021-11-25,Executive,Environmental Safety Advisor,2010-06-05,Willem.Vasco-Pais@gastech.com.kronos,NaN,NaN,NaT
44,Sanjorge Jr.,Sten,1948-08-29,Tethys,Male,Tethys,BirthNation,1948-08-29,Tethys,2013-04-21,2023-04-20,Executive,President/CEO,1990-11-09,Sten.Sanjorge Jr.@gastech.com.kronos,NaN,NaN,NaT
45,Barranco,Ingrid,1961-10-26,Tethys,Female,Tethys,BirthNation,1961-10-26,Tethys,2013-07-01,2023-06-30,Executive,SVP/CFO,2003-09-30,Ingrid.Barranco@gastech.com.kronos,NaN,NaN,NaT
46,Campo-Corrente,Ada,1951-11-26,Tethys,Female,Tethys,BirthNation,1951-11-26,Tethys,2013-07-14,2023-07-13,Executive,SVP/CIO,2004-10-05,Ada.Campo-Corrente@gastech.com.kronos,NaN,NaN,NaT


,LastName,FirstName,BirthDate,BirthCountry,Gender,CitizenshipCountry,CitizenshipBasis,CitizenshipStartDate,PassportCountry,PassportIssueDate,PassportExpirationDate,CurrentEmploymentType,CurrentEmploymentTitle,CurrentEmploymentStartDate,EmailAddress,MilitaryServiceBranch,MilitaryDischargeType,MilitaryDischargeDate
0,Mies Haber,Ruscella,1964-04-26,Kronos,Female,Kronos,BirthNation,1964-04-26,NaN,NaT,NaT,Administration,Assistant to Engineering Group Manager,2003-04-02,Ruscella.Mies.Haber@gastech.com.kronos,ArmedForcesOfKronos,HonorableDischarge,1984-10-01
1,Fusil,Stenig,1963-04-29,Tethys,Male,Tethys,BirthNation,1963-04-29,Tethys,2006-04-03,2016-04-02,Security,Building Control,2004-02-05,Stenig.Fusil@gastech.com.kronos,TethanDefenseForceArmy,HonorableDischarge,1987-10-01
6,Resumir,Felix,1959-06-30,Tethys,Male,Tethys,BirthNation,1959-06-30,Tethys,2005-03-20,2015-03-19,Security,Security Group Manager,2003-09-15,Felix.Resumir@gastech.com.kronos,TethanDefenseForceArmy,HonorableDischarge,1997-10-01
11,Mies,Henk,1984-09-23,Kronos,Male,Kronos,BirthNation,1984-09-23,NaN,NaT,NaT,Facilities,Truck Driver,2013-07-12,Henk.Mies@gastech.com.kronos,ArmedForcesOfKronos,HonorableDischarge,2004-10-01
13,Nant,Irene,1985-03-13,Kronos,Female,Kronos,BirthNation,1985-03-13,NaN,NaT,NaT,Facilities,Truck Driver,2008-11-30,Irene.Nant@gastech.com.kronos,ArmedForcesOfKronos,GeneralDischarge,2005-10-01
18,Morluniau,Cecilia,1988-09-02,Kronos,Female,Kronos,BirthNation,1988-09-02,NaN,NaT,NaT,Facilities,Truck Driver,2009-04-30,Cecilia.Morluniau@gastech.com.kronos,ArmedForcesOfKronos,GeneralDischarge,2008-10-01
19,Bodrogi,Loreto,1989-04-17,Kronos,Male,Kronos,BirthNation,1989-04-17,NaN,NaT,NaT,Security,Site Control,2013-08-17,Loreto.Bodrogi@gastech.com.kronos,ArmedForcesOfKronos,HonorableDischarge,2008-10-01
26,Ferro,Inga,1989-06-17,Kronos,Female,Kronos,BirthNation,1989-06-17,NaN,NaT,NaT,Security,Site Control,2013-01-11,Inga.Ferro@gastech.com.kronos,ArmedForcesOfKronos,GeneralDischarge,2012-10-01
30,Pantanal,Rachel,1984-08-22,Tethys,Female,Tethys,BirthNation,1984-08-22,Tethys,2013-06-13,2023-06-12,Administration,Assistant to CIO,2013-10-01,Rachel.Pantanal@gastech.com.kronos,NaN,NaN,NaT
35,Orilla,Kare,1964-02-29,Tethys,Female,Tethys,BirthNation,1964-02-29,Tethys,2013-07-01,2023-06-30,Engineering,Drill Technician,2005-11-12,Kare.Orilla@gastech.com.kronos,NaN,NaN,NaT


next up, we are applying the flags to their relevant entries in the full dataframe of nodes

In [16]:
mask = processed_SR2['Sender'].isin(filtered_top_level['EmailAddress'])
processed_SR2.loc[mask, 'Vic_Flag'] = 1

mask = processed_SR2['Sender'].isin(suspects_df['EmailAddress'])
processed_SR2.loc[mask, '1D_Flag'] = 1

mask = processed_SR2['Receiver'].isin(suspects_df['EmailAddress'])
processed_SR2.loc[mask, '2D_Flag'] = 1

# display(processed_SR2[processed_SR2['Vic_Flag'] == 1].shape)
# display(processed_SR2[processed_SR2['1D_Flag'] == 1].shape)
# display(processed_SR2[processed_SR2['2D_Flag'] == 1].shape)


### step 5: removing the irrelevant emails to reduce processing cost and keep the database legible

this part took some trial and error, but we needed to process out the irrelevant email subjects. since we dont have the email contents, we have to rely on only the subject to find communications that may help us further identify persons of interest

we are also filtering out potential nodes that do not possess one of our flags to make sure we are focused on the right group for the limited processing capability of the graph database engine

In [17]:

processed_SR3 = processed_SR2[~((processed_SR2['Vic_Flag'] != 1) & (processed_SR2['1D_Flag'] != 1) & (processed_SR2['2D_Flag'] != 1))]
print(processed_SR3.shape)
pd.set_option('display.max_rows', None)
null_subs = ['Good morning, GasTech!', 'Daily morning announcement', 'RE: Updated Safety Policies', 'RE: Planning for staff picnic',
             'RE: Changes to travel policy', 'RE: Vacation donation request', 'RE: Employee of the month', 'RE: Equipment audit approaching',
             'RE: Traffic advisory for today', 'RE: Mandatory training', 'RE: Service anniversaries!', 'RE: Parking lottery winners',
             "RE: Don't text and drive!", 'RE: Registration open: retirement planning seminar', 'RE: Babysitting recommendations',
             'RE: Action: Virus detected on your system', 'RE: GIS update', 'Daily morning announcements', 'RE: Inspection request for site',
             'RE: Yearly numbers looking good', 'RE: List of repairs needed', 'RE: Guys night out - sorry, ladie', 'RE: Safety First!',
             'RE: 2516-00 openhole logging results', 'RE: Who took the toolbox out of my truck?', 'Funny!!', 'RE: You never get a second chance to make a first impression',
             'RE: Craft night', 'RE: Watch found', 'RE: Training opportunity', 'RE: Plants', 'RE: Downhole prediction 2522-00', 'RE: Funny!!',
             'RE: Late for meeting', 'Guys night out - sorry, ladies', 'RE: Late for meeting', 'RE: Guys night out - sorry, ladies',
             'RE: Field work rotation schedule', 'RE: Ha ha', 'RE: Suggest topics for upcoming retreat', 'RE: Question - new travel forms',
             'Inspection request for site', 'Who took the toolbox out of my truck?', 'Traffic advisory for today', 'RE: Too funy - you have to see this..',
             'RE: Catering?!?', 'RE: Updated assignments', 'Wellhead flow rate data - overpressure at well 1783-03?', 'You never get a second chance to make a first impression',
             'Dress for success - Dress to impress', 'RE: Remember Casino night!', 'Favor - borrow hedge trimmer', 'Training opportunity',
             'Too funy - you have to see this...', 'Missing sweater',

             'RE: Reminder: Refer all media inquiries', 'RE: Reorganization'

            ]

processed_SR3 = processed_SR3[~processed_SR3['Subject'].isin(null_subs)]
print(processed_SR3.shape)

min_frequency = 4
search_string = 'RE: '
value_counts = processed_SR3['Subject'].value_counts()

low_freq_values = value_counts[value_counts < min_frequency].index

processed_SR3 = processed_SR3[~processed_SR3['Subject'].isin(low_freq_values)]

print(processed_SR3.shape)

display(processed_SR3.Subject.value_counts())
# display(processed_SR3)

(2727, 7)
(654, 7)
(536, 7)


RE: Impact of local politics on profit margin                  36
RE: Security procedures for January 20 VIP visit               33
Route suggestion for next shift                                27
RE: Union meeting                                              26
RE: Notification of outside visit                              26
RE: Be Careful!                                                25
RE: Managing perceptions about the IPO                         24
RE: Resolution of incident                                     21
RE: Facilities preparations for VIP visit                      20
RE: Route suggestion for next shift                            20
RE: FW: ARISE - Inspiration for Defenders of Kronos            16
RE: Wellhead flow rate data - overpressure at well 1783-03?    15
Does anyone have ...                                           14
RE: Can someone cover for me next week?                        14
RE: Impact of Kronos politics on upcoming rollout              12
RE: Cautio

in doing this, we finally get what will ostensibly constitute a list of our edges for the graph database, making sure to only keep the relevant email titles that might pertain to shady activity or be indicative of such activity if shared with other supporting evidence


### step 6: building the node queries

okay! finally here! now we are using a string template to build the nodes for the graph database. after paring everything down and isolating the important information, we are ready to build the nodes

In [18]:
df = processed_SR3.copy()
df.drop(['Sender'], axis=1)

grouped_df = df.sort_values(['Vic_Flag', '1D_Flag', '2D_Flag'], ascending=False).groupby('Receiver').first().reset_index()

query_template = (
    'MERGE (:Person {{name: "{email}", Vic_Flag: {flag1}, OneHop_Flag: {flag2}, TwoHop_Flag: {flag3}}})\n'
)

queries = grouped_df.apply(lambda row: query_template.format(
    email=row['Receiver'],
    flag1=row['Vic_Flag'],
    flag2=row['1D_Flag'],
    flag3=row['2D_Flag']
), axis=1)

cypher_query = ''.join(queries)

print(cypher_query)


MERGE (:Person {name: " Ada.Campo-Corrente@gastech.com.kronos", Vic_Flag: 1, OneHop_Flag: 0, TwoHop_Flag: 0})
MERGE (:Person {name: " Adan.Morlun@gastech.com.kronos", Vic_Flag: 0, OneHop_Flag: 1, TwoHop_Flag: 0})
MERGE (:Person {name: " Adra.Nubarron@gastech.com.kronos", Vic_Flag: 0, OneHop_Flag: 1, TwoHop_Flag: 0})
MERGE (:Person {name: " Albina.Hafon@gastech.com.kronos", Vic_Flag: 0, OneHop_Flag: 1, TwoHop_Flag: 0})
MERGE (:Person {name: " Anda.Ribera@gastech.com.kronos", Vic_Flag: 0, OneHop_Flag: 1, TwoHop_Flag: 0})
MERGE (:Person {name: " Axel.Calzas@gastech.com.kronos", Vic_Flag: 0, OneHop_Flag: 1, TwoHop_Flag: 0})
MERGE (:Person {name: " Benito.Hawelon@gastech.com.kronos", Vic_Flag: 0, OneHop_Flag: 1, TwoHop_Flag: 0})
MERGE (:Person {name: " Bertrand.Ovan@gastech.com.kronos", Vic_Flag: 0, OneHop_Flag: 1, TwoHop_Flag: 0})
MERGE (:Person {name: " Birgitta.Frente@gastech.com.kronos", Vic_Flag: 0, OneHop_Flag: 1, TwoHop_Flag: 0})
MERGE (:Person {name: " Brand.Tempestad@gastech.com.kr

note that we used 'merge' as a way to additively process this query set.

### step 7: building the edge query

at long last, we are using the node dataframe we generated above and the same process to generate the edges. 

In [19]:
edges_cypher_query = ""
s_count = 0
r_count = 0
e_count = 0

for index, row in df.iterrows():
    sender = row['Sender']
    receiver = row['Receiver']
    subject = row['Subject']

    s_var = f"s{s_count}"
    r_var = f"r{r_count}"
    e_var = f"e{e_count}"

    query = f'''
    MERGE ({s_var}:Person {{name: "{sender}"}})
    MERGE ({r_var}:Person {{name: "{receiver}"}})
    MERGE ({s_var})-[{e_var}:EMAILED]->({r_var})
    ON CREATE SET {e_var}.date = "{row['Date']}", {e_var}.subject = "{row['Subject']}"
    '''

    edges_cypher_query += query
    s_count += 1
    r_count += 1
    e_count += 1

print(edges_cypher_query)



    MERGE (s0:Person {name: "Varja.Lagos@gastech.com.kronos"})
    MERGE (r0:Person {name: "Varja.Lagos@gastech.com.kronos"})
    MERGE (s0)-[e0:EMAILED]->(r0)
    ON CREATE SET e0.date = "1/6/2014 10:28", e0.subject = "Patrol schedule changes"
    
    MERGE (s1:Person {name: "Varja.Lagos@gastech.com.kronos"})
    MERGE (r1:Person {name: " Hennie.Osvaldo@gastech.com.kronos"})
    MERGE (s1)-[e1:EMAILED]->(r1)
    ON CREATE SET e1.date = "1/6/2014 10:28", e1.subject = "Patrol schedule changes"
    
    MERGE (s2:Person {name: "Varja.Lagos@gastech.com.kronos"})
    MERGE (r2:Person {name: " Loreto.Bodrogi@gastech.com.kronos"})
    MERGE (s2)-[e2:EMAILED]->(r2)
    ON CREATE SET e2.date = "1/6/2014 10:28", e2.subject = "Patrol schedule changes"
    
    MERGE (s3:Person {name: "Varja.Lagos@gastech.com.kronos"})
    MERGE (r3:Person {name: " Inga.Ferro@gastech.com.kronos"})
    MERGE (s3)-[e3:EMAILED]->(r3)
    ON CREATE SET e3.date = "1/6/2014 10:28", e3.subject = "Patrol schedule chang